In [ ]:
# import libraries
import tensorflow as tf # for modell training
import math
import cv2 as cv
import matplotlib.pyplot as plt # to show graphical results
from matplotlib.ticker import MaxNLocator
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

In [ ]:
# own modules
import modules.config as config
from modules.ai_dataset import ai_dataset
from modules.export_trained_model import export_trained_model

In [ ]:
# import training and testing data into one class object
raw_data = ai_dataset(config.TRAINING_DATA)
#raw_data.load_data(config.TEST_DATA)
raw_data.resize(config.IMG_SIZE)
raw_data.shuffle()
raw_data.tf_ready()

In [ ]:
raw_data.get_status()

In [ ]:
# list of optimizers
optimizers = ["adadelta", "adagrad", "adam", "adamax", "ftrl", "nadam", "rmsprop", "sgd"]

In [ ]:
# split the raw data into training and test data
split_index = math.ceil(len(raw_data.get_tf_images()) * (1-config.PERCENT_TEST))

training_images = raw_data.get_tf_images()[0:split_index]
training_labels = raw_data.get_tf_labels()[0:split_index]

test_images = raw_data.get_tf_images()[split_index:]
test_labels = raw_data.get_tf_labels()[split_index:]

In [ ]:
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [ ]:
# train a model on each optimizer
for optim in optimizers:

    reset_keras()

    # print current optimizer
    print("Current Optimizer: "+optim)

    model = tf.keras.models.Sequential(name=optim)

    model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding="valid", activation="relu", input_shape=(config.IMG_SIZE, config.IMG_SIZE, 3)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding="valid"))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding="valid"))
    model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding="same", activation="relu"))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding="valid"))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(4096, activation="relu", bias_initializer="random_normal"))
    model.add(tf.keras.layers.Dense(4096, activation="relu", bias_initializer="random_normal"))
    model.add(tf.keras.layers.Dense(raw_data.get_labels_count(), activation="softmax", bias_initializer="random_normal"))


    # model.summary()

    # compile the model
    model.compile(optimizer=optim,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

    # train the model
    history = model.fit(training_images, training_labels, epochs=config.TRAINING_EPOCHS,
              validation_data=(test_images, test_labels))

    # Plot learning curve
    fig = plt.figure()
    ax1 = fig.add_subplot()
    ax2 = ax1.twinx()
    ax1.plot(history.history["accuracy"], label = "accuracy", color = "k", linestyle = "solid")
    ax1.plot(history.history["val_accuracy"], label = "val_accuracy", color = "k", linestyle = "dotted")
    ax2.plot(history.history["loss"], label = "loss", color = "k", linestyle = "dashed")
    ax2.plot(history.history["val_loss"], label = "val_loss", color = "k", linestyle = "dashdot")
    ax1.set_xlabel("Epoch")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax1.set_ylabel("Accuracy")
    ax2.set_ylabel("Loss")
    fig.legend()
    ax1.set_title("Optimizer:"+optim)

    # evaluate model
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

    # export the trained model
    export_trained_model(model, config.TRAINING_EPOCHS, test_loss, test_acc, history, fig, raw_data, config.MODEL_PATH)

In [ ]:
# export test images for predictions
if False:
    if not os.path.exists(config.MODEL_PATH+"/val_img"):
        os.makedirs(config.MODEL_PATH+"/val_img", exist_ok=True)

    for i in range(split_index, len(raw_data.get_tf_images())):
        cv.imwrite(config.MODEL_PATH+"/val_img/"+str(i)+"_"+raw_data.get_label(i)+".png",raw_data.get_image(i))

In [ ]:
    if False:
        time.sleep(60)
        os.system('shutdown -s')